In [ ]:
!pip3 install simpletransformers

     |████████████████████████████████| 215kB 8.4MB/s 
     |████████████████████████████████| 71kB 6.9MB/s 
     |████████████████████████████████| 1.3MB 16.6MB/s 
     |████████████████████████████████| 7.4MB 16.0MB/s 
     |████████████████████████████████| 1.7MB 50.7MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 317kB 46.2MB/s 
     |████████████████████████████████| 2.9MB 47.2MB/s 
     |████████████████████████████████| 1.1MB 40.6MB/s 
     |████████████████████████████████| 890kB 39.9MB/s 
     |████████████████████████████████| 133kB 49.8MB/s 
     |████████████████████████████████| 6.7MB 38.7MB/s 
     |████████████████████████████████| 163kB 55.2MB/s 
     |████████████████████████████████| 102kB 11.4MB/s 
     |████████████████████████████████| 4.4MB 41.2MB/s 
     |████████████████████████████████| 112kB 56.5MB/s 
     |████████████████████████████████| 122kB 44.4MB/s 
     |████████████████████████████████| 102kB 11.8MB/

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import *
from sklearn.model_selection import *
from scipy.special import softmax
import re
from sklearn.model_selection import StratifiedKFold
from simpletransformers.classification import MultiLabelClassificationModel
from simpletransformers.classification.classification_model import ClassificationModel
from sklearn.metrics import mean_squared_error as mse
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt
from sklearn.utils import class_weight

from sklearn.preprocessing import LabelEncoder
import pickle

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [ ]:
df_train = pd.read_csv("/content/drive/My Drive/AVNLP3/Train/Train.csv")
df_test = pd.read_csv("/content/drive/My Drive/AVNLP3/Test/Test.csv")
df_sub = pd.read_csv("/content/drive/My Drive/AVNLP3/SampleSubmission.csv")

In [ ]:
df_train['ABSTRACT'] = df_train['ABSTRACT'].apply(lambda x:' '.join(re.sub(r"[^A-Za-z0-9]"," ",x).split()).lower())
df_test['ABSTRACT'] = df_test['ABSTRACT'].apply(lambda x: ' '.join(re.sub(r"[^A-Za-z0-9]"," ",x).split()).lower())

In [ ]:
df_test

,id,ABSTRACT,Computer Science,Mathematics,Physics,Statistics
0,9409,fundamental frequency f0 approximation from po...,0,0,0,1
1,17934,this large scale study consisting of 24 5 mill...,1,0,0,1
2,16071,we present a stability analysis of the plane c...,0,0,1,0
3,16870,we construct finite time blow up solutions to ...,0,1,0,0
4,10496,planetary nebulae pne constitute an important ...,0,0,1,0
...,...,...,...,...,...,...
5997,11506,a first step inside constructing the machine l...,0,0,0,1
5998,3418,a focus of this paper was to quantify measures...,1,0,0,0
5999,7369,as autonomous vehicles become an every day rea...,1,0,0,0
6000,8421,a hamiltonian monte carlo hmc method has been ...,0,0,0,1


In [ ]:
df_temp = pd.concat([df_train,df_test])
df_temp['len'] = df_temp['ABSTRACT'].apply(lambda x: len(x.split()))
df_temp['len'].describe()

count    20006.000000
mean       164.821903
std         63.186502
min          7.000000
25%        119.000000
50%        160.000000
75%        206.000000
max        668.000000
Name: len, dtype: float64

In [ ]:
df_temp['len'].quantile(0.999)

341.0

In [ ]:
len(df_train.columns[6:].to_list())

25

In [ ]:
#Preparing data for multilabel classification
row_list = []
list_cols = df_train.columns[6:].to_list()
for index, row in df_train.iterrows():
  my_list = []
  for i in list_cols:
    my_list.append(row[i])
  row_list.append(my_list)
print(row_list)

[[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

In [ ]:
df_train['labels'] = row_list

In [ ]:
row_list = []
list_cols = df_train.columns[6:].to_list()
for index, row in df_train.iterrows():
  my_list = [row['Computer Science'],row['Mathematics'],row['Physics'],row['Statistics']]
  row_list.append(my_list)
print(row_list)
df_train['to_stratify'] = row_list

[[0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 1], [1, 1, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [1, 0, 0, 1], [0, 1, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 0], [1, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 1], [1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 1], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 1], [1, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [1, 0, 0, 1], [1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 1], [0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [1, 0, 0, 1], [1, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [1, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0], [1, 0

In [ ]:
new_abstract = []
for index, row in df_train.iterrows():
  vals = []
  if row['Computer Science'] == 1:
    vals.append('Computer Science')
  if row['Mathematics'] == 1:
    vals.append('Mathematics')
  if row['Physics'] == 1:
    vals.append('Physics')
  if row['Statistics'] == 1:
    vals.append('Statistics')
  final_text = ' '.join(vals)
  new_abstract.append(row['ABSTRACT']+' Topic is equal to '+final_text)
df_train['new_abstract'] = new_abstract

In [ ]:
new_abstract = []
for index, row in df_test.iterrows():
  vals = []
  if row['Computer Science'] == 1:
    vals.append('Computer Science')
  if row['Mathematics'] == 1:
    vals.append('Mathematics')
  if row['Physics'] == 1:
    vals.append('Physics')
  if row['Statistics'] == 1:
    vals.append('Statistics')
  final_text = ' '.join(vals)
  new_abstract.append(row['ABSTRACT']+' Topic is equal to '+final_text)
df_test['new_abstract'] = new_abstract

In [ ]:
labels = [0 for i in range(0,25)]

In [ ]:
X_train = pd.DataFrame()
X_train['text'] = df_train['new_abstract']
X_train['labels'] = df_train['labels']

test_temp = pd.DataFrame()
test_temp['text'] = df_test['new_abstract']
# test_temp['labels'] = 0
# ￼3 Replies

# test_temp['labels'] = test_temp['labels'].astype(object) 
test_temp['labels'] = [labels for i in range(len(test_temp))]

In [ ]:
#  model = MultiLabelClassificationModel('roberta','roberta-base', num_labels=25,args={'train_batch_size':32, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 2, 'max_seq_length': 224,'manual_seed': 2,'overwrite_output_dir': True})

In [ ]:
le = LabelEncoder()
df_train['to_stratify'] = le.fit_transform(df_train['to_stratify'].astype(str))

In [ ]:
def return_whole_vals(x):
  fin = []
  for i in x:
    if i > 0.30:
      fin.append(1)
    else:
      fin.append(0)
  return fin

In [ ]:
#allenai/scibert_scivocab_uncased  5FCV  78.30,thresh 0.31 seqlen 224,seqlen 350,thresh 0.33 cv 78.309
#roberta-large       5FCV  77.28,thresh 0.36 seqlen 350 

#best_scibert 4epoch 5f thresh 0.35 cv 0.8018

In [ ]:
n_folds=8
folds=StratifiedKFold(n_splits=n_folds,shuffle=True,random_state=22)
avg_cv = []
val_idxs = []
oofs = []
import pickle
final_preds =np.zeros([len(test_temp),25]) 
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train.values,df_train['to_stratify'].values)):
        print("Fold {}".format(fold_))
        X_trn = X_train.iloc[trn_idx]
        X_val = X_train.iloc[val_idx]
        model = MultiLabelClassificationModel('xlnet','xlnet-large-cased', num_labels=25,args={'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 4, 'max_seq_length': 350,'manual_seed': 2,'overwrite_output_dir': True})
        model.train_model(X_trn)
        result, model_outputs, wrong_predictions = model.eval_model(X_val)
        oofs.append(model_outputs)
        preds = [return_whole_vals(x) for x in model_outputs]
        result, model_outputs, wrong_predictions = model.eval_model(test_temp)
        avg_cv.append(f1_score(y_pred=preds,y_true=list(X_val['labels'].values),average='micro'))
        final_preds += model_outputs
        val_idxs.append(val_idx)
     
final_preds = final_preds/n_folds
with open(f"/content/drive/My Drive/AVNLP3/oofs_7f/final_preds_xlnet8f.pkl",'wb') as f:
    pickle.dump(final_preds,f)
with open(f"/content/drive/My Drive/AVNLP3/oofs_7f/oofs_xlnet_8F.pkl",'wb') as f:
    pickle.dump(oofs,f)
# with open(f"/content/drive/My Drive/AVNLP3/oofs_7f/val_idx_8F.pkl",'wb') as f:
#     pickle.dump(val_idxs,f)
print(sum(avg_cv)/n_folds)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=8.
  % (min_groups, self.n_splits)), UserWarning)


Fold 0


/usr/local/lib/python3.6/dist-packages/transformers/configuration_xlnet.py:212: FutureWarning: This config doesn't use attention memories, a core feature of XLNet. Consider setting `mem_len` to a non-zero value, for example `xlnet = XLNetLMHeadModel.from_pretrained('xlnet-base-cased'', mem_len=1024)`, for accurate training performance as well as an order of magnitude faster inference. Starting from version 3.5.0, the default parameter will be 1024, following the implementation in https://arxiv.org/abs/1906.08237
  FutureWarning,


Some weights of the model checkpoint at xlnet-large-cased were not used when initializing XLNetForMultiLabelSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing XLNetForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForMultiLabelSequenceClassification were not initialized from the model checkpoint at xlnet-large-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream t

/usr/local/lib/python3.6/dist-packages/transformers/modeling_xlnet.py:298: UserWarning: Mixed memory format inputs detected while calling the operator. The operator will output contiguous tensor even if some of the inputs are in channels_last format. (Triggered internally at  /pytorch/aten/src/ATen/native/TensorIterator.cpp:918.)
  attn_score = (ac + bd + ef) * self.scale
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:231: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Buffered data was truncated after reaching the output size limit.

In [ ]:
avg_cv

NameError: ignored

In [ ]:
sum(avg_cv)/8

0.8024220168643463

In [ ]:
def return_whole_vals(x,y):
  fin = []
  for i in x:
    if i > y:
      fin.append(1)
    else:
      fin.append(0)
  return fin

In [ ]:
# with open(f"/content/drive/My Drive/AVNLP3/oofs/preds__xlnet_large_ep40.pkl",'rb') as f:
#   model_1 = pickle.load(f)
# with open(f"/content/drive/My Drive/AVNLP3/oofs/preds__xlnet_large_ep41.pkl",'rb') as f:
#   model_2 = pickle.load(f)
# with open(f"/content/drive/My Drive/AVNLP3/oofs/preds__xlnet_large_ep42.pkl",'rb') as f:
#   model_3 = pickle.load(f)
# with open(f"/content/drive/My Drive/AVNLP3/oofs/preds__xlnet_large_ep43.pkl",'rb') as f:
#   model_4 = pickle.load(f)
# with open(f"/content/drive/My Drive/AVNLP3/oofs/preds__xlnet_large_ep44.pkl",'rb') as f:
#   model_5 = pickle.load(f)

# with open(f"/content/drive/My Drive/AVNLP3/oofs/val_idx_fold_5F_0.pkl",'rb') as f:
#   val_1 = pickle.load(f)
# with open(f"/content/drive/My Drive/AVNLP3/oofs/val_idx_fold_5F_1.pkl",'rb') as f:
#   val_2 = pickle.load(f)
# with open(f"/content/drive/My Drive/AVNLP3/oofs/val_idx_fold_5F_2.pkl",'rb') as f:
#   val_3 = pickle.load(f)
# with open(f"/content/drive/My Drive/AVNLP3/oofs/val_idx_fold_5F_3.pkl",'rb') as f:
#   val_4 = pickle.load(f)
# with open(f"/content/drive/My Drive/AVNLP3/oofs/val_idx_fold_5F_4.pkl",'rb') as f:
#   val_5 = pickle.load(f)

with open(f"/content/drive/My Drive/AVNLP3/oofs_7f/oofs_xlnet_8F.pkl",'rb') as f:
  oofs = pickle.load(f)
with open(f"/content/drive/My Drive/AVNLP3/oofs_7f/val_idx_8F.pkl",'rb') as f:
  val_idxs = pickle.load(f)

In [ ]:
oofs[0]

array([[0.00479126, 0.02865601, 0.01678467, ..., 0.00225258, 0.00462723,
        0.00844574],
       [0.00388527, 0.00226974, 0.05865479, ..., 0.00374985, 0.00346947,
        0.00874329],
       [0.00415039, 0.01628113, 0.0049057 , ..., 0.0171051 , 0.00930023,
        0.00287819],
       ...,
       [0.01242828, 0.00867462, 0.00348282, ..., 0.02433777, 0.01119995,
        0.00391388],
       [0.00163555, 0.01248169, 0.00725937, ..., 0.0026722 , 0.00168133,
        0.00176144],
       [0.00233269, 0.00568771, 0.30493164, ..., 0.00369263, 0.00298119,
        0.00291252]])

In [ ]:
final_score = []
for index, (i,j) in enumerate(zip(oofs,val_idxs)):
  preds = [return_whole_vals(x,0.38) for x in i]
  final_score.append(f1_score(y_pred=preds,y_true=list(X_train['labels'].iloc[j].values),average='micro'))
print(sum(final_score)/8)
print(final_score)

0.7933410991651476
[0.7975091260468113, 0.7816677275620624, 0.801961202302281, 0.7932260087810998, 0.8004228329809726, 0.7892187829016635, 0.7797695262483996, 0.8029535864978904]


In [ ]:
models = [[model_1,val_1],[model_2,val_2],[model_3,val_3],[model_4,val_4],[model_5,val_5]]

final_score = []
for i in models:
  preds = [return_whole_vals(x,0.40) for x in i[0]]
  final_score.append(f1_score(y_pred=preds,y_true=list(X_train['labels'].iloc[i[1]].values),average='micro'))

print(sum(final_score)/5)
print(final_score)

NameError: ignored

In [ ]:

def return_whole_vals(x):
  fin = []
  for i in x:
    if i > 0.38:
      fin.append(1)
    else:
      fin.append(0)
  return fin

In [ ]:
with open(f"/content/drive/My Drive/AVNLP3/oofs_7f/final_preds_xlnet8f.pkl",'rb') as f:
  final_preds = pickle.load(f)

In [ ]:
preds = [return_whole_vals(x) for x in final_preds]

In [ ]:
list_cols

['Analysis of PDEs',
 'Applications',
 'Artificial Intelligence',
 'Astrophysics of Galaxies',
 'Computation and Language',
 'Computer Vision and Pattern Recognition',
 'Cosmology and Nongalactic Astrophysics',
 'Data Structures and Algorithms',
 'Differential Geometry',
 'Earth and Planetary Astrophysics',
 'Fluid Dynamics',
 'Information Theory',
 'Instrumentation and Methods for Astrophysics',
 'Machine Learning',
 'Materials Science',
 'Methodology',
 'Number Theory',
 'Optimization and Control',
 'Representation Theory',
 'Robotics',
 'Social and Information Networks',
 'Statistics Theory',
 'Strongly Correlated Electrons',
 'Superconductivity',
 'Systems and Control',
 'labels']

In [ ]:
sub_df = pd.DataFrame(preds,columns=list_cols[0:25])

In [ ]:
sub_df['id']=df_sub['id']

In [ ]:
sub_df.to_csv('xlnet8folds.csv',index=False)

In [ ]:
#Scibert
[0.7791277656989386,
 0.7772727272727272,
 0.787343069433769,
 0.7842023748064018,
 0.78]